# Can you eat more pizza than your siblings?

## Riddler Express

> You and your two older siblings are sharing two extra-large pizzas and decide to cut them in an unusual way. You overlap the pizzas so that the crust of one touches the center of the other (and vice versa since they are the same size). You then slice both pizzas around the area of overlap. Two of you will each get one of the crescent-shaped pieces, and the third will get both of the football-shaped cutouts.
> 
> Which should you choose to get more pizza: one crescent or two footballs?

## Riddler Classic

> Congratulations! The Acme Axegrinders, which you own, are the regular season champions of the National Squishyball League (NSL). Your team will now play a championship series against the Boondocks Barbarians, which had the second-best regular season record. You feel good about Acme’s chances in the series because Acme won exactly 60 percent of the hundreds of games it played against Boondocks this season. (The NSL has an incredibly long regular season.) The NSL has two special rules for the playoffs:
> 
> 1. The owner of the top-seeded team (i.e., you) gets to select the length of the championship series in advance of the first game, so you could decide to play a single game, a best two out of three series, a three out of five series, etc., all the way up to a 50 out of 99 series.
> 2. The owner of the winning team gets \$1 million minus \$10,000 for each of the victories required to win the series, regardless of how many games the series lasts in total. Thus, if the top-seeded team’s owner selects a single-game championship, the winning owner will collect \$990,000. If he or she selects a 4 out of 7 series, the winning team’s owner will collect \$960,000. The owner of the losing team gets nothing.
>
> Since Acme has a 60 percent chance of winning any individual game against Boondocks, Rule 1 encourages you to opt for a very long series to improve Acme’s chances of winning the series. But Rule 2 means that a long series will mean less winnings for you if Acme does take the series.
> 
> How long a series should you select in order to maximize your expected winnings? And how much money do you expect to win?

Let's first define some terms.

- $S$: is a series
- $n$: the number of wins required for a series $S_n$.
- $N$: the maximum number of games a series $S_n$ can go.  $N = 2n-1$
- $m$: the number of total games a particular series went before a winner was determined.
- $w$: the number of wins in a particular series.
- $P_n$: the probability of winning a series requiring $n$ wins.
- $P_{n,m}$: the probability of winning a series requiring $n$ wins in a total of $m$ games.

A series $S_n$ can be won in anywhere from $n$ to $2n-1$ games.  We can calculate the probability of winning the overall series by adding up the probabilities of each series run length.

$P_n = \sum\limits_{m=n}^{2n-1}P_{n,m}$

But we need to take care that we don't overcount.  That is, for a series $S_n$ that is won in $n+1$ games, we need to remove the combinations of wins of $n$ games (because that series would be ended in $n$ games).  In general, when calculating the probability of a series going to $m$ games, we need to remove the combinations of wins where $n \lt m$.

So there are a number of steps to calculate the overall probability.
1. The probability of $n$ wins in $m$ games.  This takes into consideration the combinations of wins and losses as well as the probabilities of wins and losses.
2. The probability of $n$ wins in a series of exactly $m$ games.  This means the series was not concluded in less than $m$ games, so the probabilities of shorter series must be removed from the combinations.

In [7]:
import scipy.misc

def win_probability(w, m, p=0.6):
    """Return the probability of `w` wins out of `m` games with a base win probability of `p`.
    The wins and losses can occur in any order.
    """
    assert w <= m, 'the number of wins must be less than or equal to the number of games'
    losses = m - w
    n_combinations = scipy.misc.comb(m, w)
    return p**w * (1-p)**losses * n_combinations

# what is the probability of 1 win out 1 game.
win_probability(1, 1)

0.59999999999999998

In [6]:
win_probability(0, 1)

0.40000000000000002

In [89]:
def series_win_probability(n, m, p=0.6, verbose=False):
    """Return the probability of `n` wins for a series of length `m` games with a base win probaility of `p`.
    The wins and losses can occur in any order, but we are removing the occurrences of wins that result
    in a shorter series.
    The shortest the series can be is `n` games while the longest is `2n-1`.
    """
    assert n <= m, 'the number of wins {n} must be less than or equal to the number of games in the series {m}'.format(n=n, m=m)
    assert m <= 2*n - 1, 'the series length cannot be greater than 2n-1'
    losses = m - n
    all_combinations = scipy.misc.comb(m, n)
    # lesser_combinations = sum([scipy.misc.comb(i, n) for i in range(n, m)])
    lesser_combinations = scipy.misc.comb(m-1, n)
    n_combinations = all_combinations - lesser_combinations
    prob = p**n * (1-p)**losses * n_combinations
    if verbose:
        print('{} wins out of {} games, p={}'.format(n, m, p))
        print('all combinations:    {}'.format(all_combinations))
        print('lesser combinations: {}'.format(lesser_combinations))
        print('n_combinations:      {}'.format(n_combinations))
        print('probability:         {}'.format(prob))
    return prob


series_win_probability(1, 1, p=0.6, verbose=True)

1 wins out of 1 games, p=0.6
all combinations:    1.0
lesser combinations: 0.0
n_combinations:      1.0
probability:         0.6


0.59999999999999998

In [90]:
series_win_probability(1, 1, p=0.4, verbose=True)

1 wins out of 1 games, p=0.4
all combinations:    1.0
lesser combinations: 0.0
n_combinations:      1.0
probability:         0.4


0.40000000000000002

In [91]:
N = 2
p = 0.6
for m in range(N, 2*N):
    print('\tP of {} wins in {} games: {}'.format(N, m, series_win_probability(N, m, p=p, verbose=True)))

2 wins out of 2 games, p=0.6
all combinations:    1.0
lesser combinations: 0.0
n_combinations:      1.0
probability:         0.36
	P of 2 wins in 2 games: 0.36
2 wins out of 3 games, p=0.6
all combinations:    3.0
lesser combinations: 1.0
n_combinations:      2.0
probability:         0.288
	P of 2 wins in 3 games: 0.288


In [92]:
N = 2
p = 0.4
for m in range(N, 2*N):
    print('\tP of {} wins in {} games: {}'.format(N, m, series_win_probability(N, m, p=p, verbose=True)))

2 wins out of 2 games, p=0.4
all combinations:    1.0
lesser combinations: 0.0
n_combinations:      1.0
probability:         0.16000000000000003
	P of 2 wins in 2 games: 0.16000000000000003
2 wins out of 3 games, p=0.4
all combinations:    3.0
lesser combinations: 1.0
n_combinations:      2.0
probability:         0.19200000000000003
	P of 2 wins in 3 games: 0.19200000000000003


In [93]:
N = 3
p = 0.5
print('\t{}'.format(sum([series_win_probability(N, m, p=p, verbose=True) for m in range(N, 2*N)])))
print('\t{}'.format(sum([series_win_probability(N, m, p=1-p, verbose=True) for m in range(N, 2*N)])))
print('\ntotal probability: {}'.format(
    sum([series_win_probability(N, m, p=p, verbose=False) for m in range(N, 2*N)]) + 
    sum([series_win_probability(N, m, p=1-p, verbose=False) for m in range(N, 2*N)])
))

3 wins out of 3 games, p=0.5
all combinations:    1.0
lesser combinations: 0.0
n_combinations:      1.0
probability:         0.125
3 wins out of 4 games, p=0.5
all combinations:    4.0
lesser combinations: 1.0
n_combinations:      3.0
probability:         0.1875
3 wins out of 5 games, p=0.5
all combinations:    10.0
lesser combinations: 4.0
n_combinations:      6.0
probability:         0.1875
	0.5
3 wins out of 3 games, p=0.5
all combinations:    1.0
lesser combinations: 0.0
n_combinations:      1.0
probability:         0.125
3 wins out of 4 games, p=0.5
all combinations:    4.0
lesser combinations: 1.0
n_combinations:      3.0
probability:         0.1875
3 wins out of 5 games, p=0.5
all combinations:    10.0
lesser combinations: 4.0
n_combinations:      6.0
probability:         0.1875
	0.5

total probability: 1.0


In [99]:
N = 50
p = 0.6
print('\t{}'.format(sum([series_win_probability(N, m, p=p, verbose=False) for m in range(N, 2*N)])))
print('\t{}'.format(sum([series_win_probability(N, m, p=1-p, verbose=False) for m in range(N, 2*N)])))
print('\ntotal probability: {}'.format(
    sum([series_win_probability(N, m, p=p, verbose=False) for m in range(N, 2*N)]) + 
    sum([series_win_probability(N, m, p=1-p, verbose=False) for m in range(N, 2*N)])
))

	0.978069557869915
	0.021930442130085215

total probability: 1.0000000000000002


First let's define some terms.

- $n$: the number of wins required to win the series.
- $N$: the maximum number of games the series can run.  $N = 2n-1$
- $m$: the number of games a series went in order to determine a winner.
- $w$: the number of wins in a particular series.

Let's imagine that a series which requires $n$ wins will always play to the full $2n - 1$ games.  This will mean that many of the possible sequences have more than $n$ wins.  We can then limit ourselves to the first $m$ games in each sequence, where $n \leq m \leq 2n-1$.  This allows us to calculate the probability that a series will go to $m$ games.

Each sequence of length $2n-1$ with $w$ wins has probability

$P_w = p^w(1-p)^{N-w}

First, let's calculate the probability of $n$ wins out of (at most) $2n - 1$ games in the series, given the probability of winning any single game is $p = 0.6$.

Since the series is over once either team wins $n$ games, this could be done in anywhere from $n$ to $2n - 1$ games (inclusive).  And for a series of length $m$ (where $n \leq m \leq 2n - 1$) the number of ways we can order the wins is combinatorial.

$N = {m \choose n} = \dfrac{m!}{n!(m-n)!}$

So the probability of winning $n$ games and losing $m - n$ games is

$P_{w=n,l=m-n} = p^n (1-p)^{m-n} {m \choose n}$

That is, the product of $n$ wins times $m - n$ losses and ${m \choose n}$ ways to arrange those wins and losses.

There is also the question of the probability that the series of $n$ wins would be decided in $m$ games.  This is a counting problem where the numberator is the number of ways this particular series 

The total probability of winning the series is the sum of the probabilities of all possible series lengths.

$P = \sum\limits_{m=n}^{2n-1}P_m$



In [ ]:
import scipy.misc

def win_loss_probability(wins, losses, p=0.6):
    """Get the probability of `wins` and `losses` with a base win probability of `p`.
    The wins and losses can occur in any order.
    """
    N = wins + losses
    nCr = scipy.misc.comb(N, wins)
    return p**wins * (1-p)**losses * nCr

# what is the probability of 1 win and 0 losses?
win_loss_probability(1, 0)

In [29]:
# what is the probability of 0 wins and 1 loss?
win_loss_probability(0, 1)

0.40000000000000002

In [31]:
def series_win_probability(wins, p=0.6):
    """Get the probability of winning a series that requires `wins` number of wins.
    This could be out of a total of anywhere from $wins$ to $2*wins - 1$ games.
    """
    return sum([win_loss_probability(wins, losses, p=p) for losses in range(0, wins)])

series_win_probability(wins=5)

1.0551720959999999

In [32]:
list(range(0, 4))

[0, 1, 2, 3]